In [8]:
import scrapy
import requests
import asyncio
from scrapy.crawler import Crawler
import json
from urllib.parse import quote

In [ ]:
class Configurations:
    def __init__(self, proxy, urls) -> None:
        self.proxy: str = proxy,
        self.urls: list = urls
        pass
    
       
    def create_client(self) -> requests.Session:
        if self.proxy is None:
            raise Exception("no proxy available")
        proxies = {"http": self.proxy}
        headers = {
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:137.0) Gecko/20100101 Firefox/137.0"
        }
        client = requests.Session()
        client.proxies.update(proxies)
        return client
    
    
    def check_status(self) -> None:
        client = create_client()
        for url in self.urls:
            response = client.get(url)
            return print(response.status_code)
   

        
class NYTimesSpider(scrapy.Spider, Configurations):
    name = "nytimesspider"
    allowed_domains = ["nytimes.com"]
    start_urls = ["https://nytimes.com/section/business/media"]

    def start_requests(self, cursor: str = None):
        operation_name = "CollectionsQuery"
        variables = {
    "id":"/section/business/media",
    "first":10,
    "exclusionMode":"HIGHLIGHTS_AND_EMBEDDED",
    "isFetchMore":False,
    "isTranslatable":False,
    "isEspanol":False,
    "highlightsListUri":"nyt://per/personalized-list/__null__",
    "highlightsListFirst":0,
    "hasHighlightsList":False,
    "cursor": cursor # None when Opening the page
        }

        extension= {"persistedQuery":
                          {"version":1,"sha256Hash":"8334262659d77fc2166184bf897e6d139e437af3a9b84d0c020d3dfcb0f177b8"}
                          }
        var_query = quote(json.dumps(variables))
        extension_query = quote(json.dumps(variables))


        url = f"https://samizdat-graphql.nytimes.com/graphql/v2?operation_name={operation_name}&variables={var_query}&extension={extension_query}"


        yield scrapy.Request(
            url=url,
            headers={"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:137.0) Gecko/20100101 Firefox/137.0"},
            callback=self.parse 
            )



    def parse(self, response):
        data = response.json()
        
        collection = data['data']['legacyCollection']['collectionsPage']
        articles = collection['stream']['edges']

        for article in articles:
            yield {
                'headline':article['node']['headline']['default'],  #Need to get text which is in default="headline"
                'summary': article['node']['summary'],
                'url': article['node']['url'],
                'News Source': article['node']['_typename']
            }
        
        #Now we need to parse through the pages
        #The end paramater will be the new start parameter
        start_cursor = collection['stream']['pageInfo']['endCursor']
        if start_cursor:
            yield self.make_next_request(start_cursor)
        
    def make_next_request(self, new_cursor):
        start_request(new_cursor)
        
        
        
    
    

    
    
    
def main():
    example = Extractor(["https://www.nytimes.com/media"])
    process = CrawlerProcess(example.create_client.headers)
    process.crawl(NYTimesSpider)
    process.start()
    


if __name__ == "__main__":
    main()
        

Building an API query:

https://samizdat-graphql.nytimes.com/graphql/v2?
operationName=CollectionsQuery&
variables={
    "id":"/section/business/media",
    "first":10,
    "exclusionMode":"HIGHLIGHTS_AND_EMBEDDED",
    "isFetchMore":false,
    "isTranslatable":false,
    "isEspanol":false,
    "highlightsListUri":"nyt://per/personalized-list/__null__",
    "highlightsListFirst":0,
    "hasHighlightsList":false,
    "cursor":"YXJyYXljb25uZWN0aW9uOjIwOQ=="}
    &extensions=
    {"persistedQuery":{"version":1,"sha256Hash":"8334262659d77fc2166184bf897e6d139e437af3a9b84d0c020d3dfcb0f177b8"}}

https://samizdat-graphql.nytimes.com/graphql/v2?operationName=CollectionsQuery&variables=%7B%22id%22%3A%22%2Fsection%2Fbusiness%2Fmedia%22%2C%22first%22%3A10%2C%22exclusionMode%22%3A%22HIGHLIGHTS_AND_EMBEDDED%22%2C%22isFetchMore%22%3Afalse%2C%22isTranslatable%22%3Afalse%2C%22isEspanol%22%3Afalse%2C%22highlightsListUri%22%3A%22nyt%3A%2F%2Fper%2Fpersonalized-list%2F__null__%22%2C%22highlightsListFirst%22%3A0%2C%22hasHighlightsList%22%3Afalse%2C%22cursor%22%3A%22YXJyYXljb25uZWN0aW9uOjIwOQ%3D%3D%22%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%228334262659d77fc2166184bf897e6d139e437af3a9b84d0c020d3dfcb0f177b8%22%7D%7D

In [6]:
import base64


decoded1 = base64.b64decode("YXJyYXljb25uZWN0aW9uOjE5").decode('utf-8')
decoded2 = base64.b64decode("YXJyYXljb25uZWN0aW9uOjM5").decode('utf-8')
print(decoded1)
print(decoded2)

arrayconnection:19
arrayconnection:39


In [ ]:
#Example query
operation_name = "CollectionsQuery"
variables = {
    "id":"/section/business/media",
    "first":10,
    "exclusionMode":"HIGHLIGHTS_AND_EMBEDDED",
    "isFetchMore":False,
    "isTranslatable":False,
    "isEspanol":False,
    "highlightsListUri":"nyt://per/personalized-list/__null__",
    "highlightsListFirst":0,
    "hasHighlightsList":False,
    "cursor":"YXJyYXljb25uZWN0aW9uOjE5"
}
extension= {"persistedQuery":
                          {"version":1,"sha256Hash":"8334262659d77fc2166184bf897e6d139e437af3a9b84d0c020d3dfcb0f177b8"}
                          }
proxies = {"http": "103.116.7.1"}

var_query = quote(json.dumps(variables))
extension_query = quote(json.dumps(extension))


url = f"https://samizdat-graphql.nytimes.com/graphql/v2?operation_name={operation_name}&variables={var_query}&extension={extension_query}"

r = requests.get(
    url=url, 
    headers={"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:137.0) Gecko/20100101 Firefox/137.0"}, 
    proxies=proxies
    )
r.json()

'https://samizdat-graphql.nytimes.com/graphql/v2?operation_name=CollectionsQuery&variables=%7B%22id%22%3A%20%22/section/business/media%22%2C%20%22first%22%3A%2010%2C%20%22exclusionMode%22%3A%20%22HIGHLIGHTS_AND_EMBEDDED%22%2C%20%22isFetchMore%22%3A%20false%2C%20%22isTranslatable%22%3A%20false%2C%20%22isEspanol%22%3A%20false%2C%20%22highlightsListUri%22%3A%20%22nyt%3A//per/personalized-list/__null__%22%2C%20%22highlightsListFirst%22%3A%200%2C%20%22hasHighlightsList%22%3A%20false%2C%20%22cursor%22%3A%20%22YXJyYXljb25uZWN0aW9uOjE5%22%7D&extension=%7B%22id%22%3A%20%22/section/business/media%22%2C%20%22first%22%3A%2010%2C%20%22exclusionMode%22%3A%20%22HIGHLIGHTS_AND_EMBEDDED%22%2C%20%22isFetchMore%22%3A%20false%2C%20%22isTranslatable%22%3A%20false%2C%20%22isEspanol%22%3A%20false%2C%20%22highlightsListUri%22%3A%20%22nyt%3A//per/personalized-list/__null__%22%2C%20%22highlightsListFirst%22%3A%200%2C%20%22hasHighlightsList%22%3A%20false%2C%20%22cursor%22%3A%20%22YXJyYXljb25uZWN0aW9uOjE5%22%7D'

In [ ]:
extension_query = quote(json.dumps(extension))  # Fix the variable
proxies = {"http": "http://xxxxxxxx", "https": "http://xxxxxxxx"}  # Proper format

r = requests.get(
    url=url, 
    headers={"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:137.0) Gecko/20100101 Firefox/137.0"},
    proxies=proxies
)
r.raise_for_status()  # Check for HTTP errors
data = r.json()